In [26]:
import tarfile
import pandas as pd
import io
import datetime

def toDatetime(dt):
    return datetime.datetime.strptime(dt, "%Y-%m-%d %H:%M")

def ungzToDataframe(tarGzFile,columnNames):
    
    '''
    無法直接用 pandas 的read_csv(...compression='gzip') , 欄位會跑掉, 尚未找到好方法進行異常處理.
    df = pd.read_csv('M03A_20190227.tar.gz', compression='gzip', header=None, sep=',', quotechar='"', error_bad_lines=False)

    所以考慮土方法 解壓縮成一個一個 csv 再處理
    '''
    
    result=pd.DataFrame()
    tar = tarfile.open(tarGzFile, "r:gz")
    for member in tar.getmembers():
        f = tar.extractfile(member)

        if f is not None:
            content = f.read()
            df = pd.read_csv(io.StringIO(content.decode('utf-8')), header=None, sep=',')
            
            result=result.append(df, ignore_index=True)
    
    result.columns = columnNames
    result.TimeInterval=result.TimeInterval.apply(toDatetime)
    return result

In [27]:
columnNames = ['VehicleType','DetectionTime_O', 'GantryID_O', 'DetectionTime_D', 'GantryID_D', 'TripLength', 'TripEnd', 'TripInformation'] #M06A
#df = ungzToDataframe('M06A/M06A_20190107.tar.gz', columnNames)

In [28]:
columnNames = ['TimeInterval','GantryFrom','GantryTo', 'VehicleType', 'TravelTime', '交通量'] #M04A
#df = ungzToDataframe('M04A/M04A_20191013.tar.gz', columnNames)

In [18]:
import pymongo
import dns
client = pymongo.MongoClient("mongodb://localhost:27017")
db = client.traffic

def dataframe2Mongo(select_df, trfType):
    collection = db[trfType]  
    records = select_df.to_dict('records') # 參數 record 代表把列轉成個別物件
    collection.insert_many(records)


In [23]:
dayToInsert=['0624','0625','0626','0627','0628']

In [24]:
columnNames = ['TimeInterval','GantryId', 'Direction', 'VehicleType', '交通量'] #M03A

for d in dayToInsert:
    print('---',d)
    df = ungzToDataframe('../../../data/M03A/'+'M03A_2019'+d+'.tar.gz', columnNames)
    dataframe2Mongo(df, 'TDCS-M03A')

--- 0624
--- 0625
--- 0626
--- 0627
--- 0628


In [25]:
columnNames = ['TimeInterval','GantryFrom','GantryTo', 'VehicleType', '交通量'] #M08A
for d in dayToInsert:
    print('---',d)
    df = ungzToDataframe('../../../data/M08A/'+'M08A_2019'+d+'.tar.gz', columnNames)
    dataframe2Mongo(df, 'TDCS-M08A')


--- 0624
--- 0625
--- 0626
--- 0627
--- 0628
